In [ ]:
using Revise
using ConvexHullPricing
using DataFrames
using Plots
using JLD2
using ProgressBars, JuMP, Gurobi
const UT = ConvexHullPricing.Utilitaries
const OPT = ConvexHullPricing.Optimizer

In [ ]:
ThermalGen = ConvexHullPricing.Utilitaries.ThermalGen(
    MinRunCapacity = [6],
    MaxRunCapacity = [16],
    RampUp = [5],
    RampDown = [5],
    StartUp = [6],
    ShutDown = [6],
    UpTime = [1],
    DownTime = [1],
    NoLoadConsumption = [10],
    MarginalCost = [53],
    FixedCost = [30],
)
instance = ConvexHullPricing.Utilitaries.Instance(
    LostLoad = 3000,
    Load = [6 11 16 11],
    ThermalGen = ThermalGen
)
LP_Relax = UT.LP_Relaxation(instance)
X0 = UT.LP_Relaxation(instance)

In [ ]:
XBLM, ITBLM, FBLM = OPT.BundleLevelMethod(instance, X0, 10, 0.7)

In [ ]:
XCG, ITCG, FCG, TCG = OPT.tColumnGeneration(instance, X0, .1, 0.)

In [ ]:
XCRG, ITCRG, FCRG, TCRG = OPT.tCRG(instance, .1, 1e-6, 0)

In [ ]:
plot(FBLM, label = "BLM")
plot!(FCG, label = "CG")
plot!(FCRG, label = "CRG")